# Create Cooperative Change of Name Special Resolution

## Setup environment
Before running following environment setup snippets, ensure environment variables found in `default-bcr-business-setup-TEST` notebook contain the correct values.

In [ ]:
%run /workspaces/lear/tests/data/default-bcr-business-setup-TEST.ipynb

In [ ]:
%run /workspaces/lear/tests/data/common/legal_api_utils.ipynb
%run /workspaces/lear/tests/data/common/legal_api_utils_test_payload_data.ipynb
%run /workspaces/lear/tests/data/common/namex_api_utils.ipynb
%run /workspaces/lear/tests/data/common/auth_api_utils.ipynb

In [ ]:
import copy
import re
import json
from legal_api.core import Filing as CoreFiling
from legal_api.models import Business, Comment, Filing, RegistrationBootstrap
from legal_api.utils.datetime import date, datetime

## Input Values for filing
Please update following values to appropriate values before running subsequent code snippets!!

In [ ]:
verify_ssl=True # set to False if using proxy to debug requests

dt = datetime.utcnow()
dt_str = dt.strftime("%Y-%m-%d")

legal_type = 'CP'
filing_type = 'specialResolution'

# Replace following placeholders(<placeholder>) with appropriate values
identifier = '<placeholder>' # expected format CP1002533
change_of_name_nr_num = '<placeholder>' # change of name nr.  expected format "NR 6771356"
email = '<placeholder>'
effectiveDate = '<placeholder>' # should be in format 2022-03-01T07:01:00+00:00

certified_by = '<placeholder>'

signatory_given_name = '<placeholder>'
signatory_family_name = '<placeholder>'
signatory_additional_name = '<placeholder>' # use None if not additional name is required

resolution_text = '<placeholder>'
signing_date = '<placeholder>' # should be in format 2022-01-11
resolution_date = '<placeholder>' # should be in format 2022-01-11

# fill in applicable payment related details
# routing_slip_number is used when paying with routing slip
routing_slip_number = None # optional - should be in format '650000001'
# bcol_account_number and dat_number are used when paying with BCOL
bcol_account_number = None # optional - should be in format '109970'
dat_number = None # optional - should be in format 'C8473902'
folio_number = None # optional - should be in format '1111110000'

dt_str, legal_type, identifier, change_of_name_nr_num, email, \
signatory_given_name, signatory_family_name, signatory_additional_name, \
resolution_text, signing_date, resolution_date, \
routing_slip_number, bcol_account_number, dat_number, folio_number

## Tokens

In [ ]:
auth_token = get_auth_token(verify_ssl)
assert auth_token
# auth_token

In [ ]:
lear_token = get_lear_token(verify_ssl)
assert lear_token
# lear_token

### Get NR Info

In [ ]:
legal_name = get_nr_approved_name(change_of_name_nr_num, email, verify_ssl)
legal_name

## Get Account Info

In [ ]:

r = get_account_by_affiliated_identifier(auth_token, identifier, verify_ssl)
assert r.status_code == 200
account_info = r.json()

account_number = account_info['orgs'][0]['id']
account_number, account_info


## Create Special Resolution Filing

In [ ]:

filing_json = factory_empty_coop_special_resolution_payload()
filing_json['filing']['header']['date'] = dt_str
filing_json['filing']['header']['certifiedBy'] = certified_by
if routing_slip_number:
    filing_json['filing']['header']['routingSlipNumber'] = routing_slip_number
if bcol_account_number:
    filing_json['filing']['header']['bcolAccountNumber'] = bcol_account_number
if dat_number:
    filing_json['filing']['header']['datNumber'] = dat_number
if folio_number:
    filing_json['filing']['header']['folioNumber'] = folio_number

filing_json['filing']['changeOfName']['nameRequest']['nrNumber'] = change_of_name_nr_num
filing_json['filing']['changeOfName']['nameRequest']['legalName'] = legal_name

filing_json['filing']['specialResolution']['resolution'] = resolution_text
filing_json['filing']['specialResolution']['signingDate'] = signing_date
filing_json['filing']['specialResolution']['resolutionDate'] = resolution_date

filing_json['filing']['specialResolution']['signatory']['givenName'] = signatory_given_name
filing_json['filing']['specialResolution']['signatory']['familyName'] = signatory_family_name
if signatory_additional_name:
    filing_json['filing']['specialResolution']['signatory']['additionalName'] = signatory_additional_name
else:
    del filing_json['filing']['specialResolution']['signatory']['additionalName']

# print(json.dumps(filing_json, indent=4))

business = Business.find_by_identifier(identifier)

filing = Filing()
filing.effective_date=datetime.fromisoformat(effectiveDate)
filing.business_id = business.id
filing._filing_json = filing_json
filing._filing_type = filing_type
filing.filing_date = datetime.fromisoformat(effectiveDate)


# Override the state setting mechanism
filing.skip_status_listener = True
filing._status = Filing.Status.PENDING.value
filing.source = Filing.Source.LEAR.value

# print(json.dumps(filing.json, indent=4))

In [ ]:
filing.save()
filing_id = filing.id
filing_id

## Manually place on the Q for processing

In [ ]:
processed_filing = Filing.find_by_id(filing_id)
processed_filing.status

In [ ]:
mailing_address = business.mailing_address.one_or_none()
mailing_address

In [ ]:
filing_types = [{ 'filingTypeCode': 'SPRLN', 'priority': False, 'waiveFees': False },
                { 'filingTypeCode': 'OTCON', 'priority': False, 'waiveFees': False }]

invoice = create_invoice(business=business,
                         corp_type=legal_type,
                         mailing_address=mailing_address,
                         filing_types=filing_types,
                         routing_slip_number=routing_slip_number,
                         bcol_account_number=bcol_account_number,
                         dat_number=dat_number,
                         folio_number=folio_number)

invoice

In [ ]:
r = submit_filing_invoice(lear_token, invoice, verify_ssl)
print(f'status_code: {r.status_code}')
if r.status_code != 201:
    print(f'status code: {r.status_code}, response: {r.json()}')
assert r.status_code == 201

payment_id = r.json().get('id')

processed_filing._payment_token = payment_id
processed_filing.payment_status_code = r.json().get('statusCode', '')
processed_filing.payment_account = account_number
processed_filing.save()

payment_id, r.json()